# 04 — Демонстрация онлайн-встречи

Цель ноутбука — собрать воедино все компоненты в упрощённом прототипе:
- несколько «виртуальных» участников (разные аудиофайлы/записи);
- периодическая отправка аудио в ASR + Speaker ID;
- построение контекста беседы и промптов;
- периодическая генерация новых изображений под контекст встречи.

Здесь можно реализовать монолитный прототип (без микросервисов) и показать, как он логически распадается на сервисы из `docs/architecture.md`. 


In [ ]:
# Заготовка для монолитного прототипа онлайн-встречи

from dataclasses import dataclass, field
from typing import List, Optional


@dataclass
class Utterance:
    text: str
    speaker_id: str
    timestamp: float


@dataclass
class MeetingState:
    utterances: List[Utterance] = field(default_factory=list)
    max_history_seconds: float = 300.0  # окно контекста ~5 минут

    def add_utterance(self, utt: Utterance):
        self.utterances.append(utt)
        # здесь позже можно добавить обрезку истории по времени

    def build_prompt(self) -> str:
        """Простейшее построение промпта по последним репликам."""
        parts = [f"[{u.speaker_id}] {u.text}" for u in self.utterances[-10:]]
        conversation = " \n".join(parts)
        prompt = (
            "An illustration summarizing the ongoing conversation between participants. "
            "Focus on the main themes and emotions. Conversation: " + conversation
        )
        return prompt


meeting_state = MeetingState()

